In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
from sklearn import datasets
iris = datasets.load_iris()
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [3]:
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.DataFrame(iris.target, columns=['species'])
df = pd.concat([X,y], axis=1)

In [4]:
# оставляем 2 класса
df = df[df['species'] != 0]
df['species'].where(df['species'] == 1, 0, inplace=True)

In [5]:
# выделяем y
y = np.array(df['species'])
df.drop(['species'], axis=1, inplace=True)

In [6]:
# нормализую фичи
columns = df.columns
scaler = StandardScaler()
df[columns[0]] = scaler.fit_transform(np.array(df[columns[0]]).reshape(-1, 1))
df[columns[1]] = scaler.fit_transform(np.array(df[columns[1]]).reshape(-1, 1))
df[columns[2]] = scaler.fit_transform(np.array(df[columns[2]]).reshape(-1, 1))
df[columns[3]] = scaler.fit_transform(np.array(df[columns[3]]).reshape(-1, 1))

In [7]:
# получаю X
X = np.array(df[columns])

In [8]:
def sigmoid(scores):
    return 1 / (1 + np.exp(-scores))

In [9]:
def loss(Y_pred, Y_label):
    return (-Y_label * np.log(Y_pred) - (1 - Y_label) * np.log(1 - Y_pred)).mean()

In [10]:
def log_like(features, target, weights):
    scores = np.dot(features, weights)
    return  -np.sum( target*scores - np.log(1 + np.exp(scores)) ) 

In [11]:
def logistic_regression(features, target, steps, learning_rate):

    weights = np.zeros(features.shape[1])
    
    for step in range(steps):
        scores = np.dot(features, weights)
        predictions = sigmoid(scores)
        error = target - predictions
        gradient = np.dot(features.T, error)
        weights += learning_rate * gradient

        if(step % 1000 == 0):
            print (loss(predictions, target))
        
    return weights

In [12]:
logistic_regression(X, y, 10000, 0.001)

0.6931471805599453
0.08671066528517198
0.07392458634698458
0.06910262175058458
0.06657721242226679
0.06503897900604519
0.06401502260202047
0.06329198573226077
0.06275943992131255
0.062354572613840406


array([ 1.28950483,  1.59245194, -5.80687679, -5.77689667])